#### Identify the url to write code that loads eBay's search result page containing sold "amazon gift card". Save the result to file. Give the file the filename "amazon_gift_card_01.htm"

In [2]:
# to load the page

from bs4 import BeautifulSoup
import requests
import time
import re

url1= "https://www.ebay.com/sch/i.html?_from=R40&_nkw=amazon+gift+card&_sacat=0&LH_Sold=1&rt=nc"
headers = {'User-Agent': 'Mozilla/5.0' }
    
page1 = requests.get(url1, headers=headers)
print(page1)
# Create a beautifulsoup object 
soup1 = BeautifulSoup(page1.text, 'lxml')

<Response [200]>


In [3]:
## to save

def saveString(html, filename="test.html"):
    try:
        file = open(filename,"w", encoding="utf-8")
        file.write(str(html))
        file.close()
    except Exception as ex:
        print('Error: ' + str(ex))


def loadString(f="test.html"):
    try:
        html = open(f, "r", encoding='utf-8').read()
        return(html)
    except Exception as ex:
        print('Error: ' + str(ex))

In [4]:
folder_to_Save = "./"

In [46]:
saveString(requests.get(url1, headers=headers).text, filename=folder_to_Save+"amazon_gift_card_01.htm")

#### Now, write a loop that will download the first 10 pages of search results. Save each of these pages to "amazon_gift_card_XX.htm" (XX = page number). 

In [5]:
pages_url_list=[]

for i in range(1,11):
    pages_url_list.append("https://www.ebay.com/sch/i.html?_from=R40&_nkw=amazon+gift+card&_sacat=0&LH_Sold=1&rt=nc&_pgn="+str(i))
    

#print(pages_url_list) 
print(len(pages_url_list))

10


In [1]:
for i in range(10):
    saveString(requests.get(pages_url_list[i], headers=headers).text, filename=folder_to_Save+"amazon_gift_card_"+ str(i+1) +".htm")
    time.sleep(10)
#each page request needs to be followed by a 10 second pause. You want your program to mimic your behavior as a human and help you make good purchasing decisions.
    

NameError: name 'saveString' is not defined

####  using loops through the pages we downloaded above, open and parse them to a Python object.

In [51]:
textpages=[]

for i in range(10):
    loadString(folder_to_Save+"amazon_gift_card_"+ str(i+1) +".htm")
    textpages.append(BeautifulSoup(loadString(folder_to_Save+"amazon_gift_card_"+ str(i+1) +".htm"), 'html.parser'))
  
print(textpages[0].text)
#print(len(textpages))

amazon gift card for sale | eBay
 Skip to main contentShop by categoryShop by categoryEnter your search keyword All CategoriesAntiques Art Baby Books & Magazines Business & Industrial Cameras & Photo Cell Phones & Accessories Clothing, Shoes & Accessories Coins & Paper Money Collectibles Computers/Tablets & Networking Consumer Electronics Crafts Dolls & Bears Entertainment Memorabilia Everything Else Gift Cards & Coupons Health & Beauty Home & Garden Jewelry & Watches Movies & TV Music Musical Instruments & Gear Pet Supplies Pottery & Glass Real Estate Specialty Services Sporting Goods Sports Mem, Cards & Fan Shop Stamps Tickets & Experiences Toys & Hobbies Travel Video Games & ConsolesAdvancedHi (Sign in to bid or buy) Daily Deals Brand Outlet Help & Contact SellWatchlistExpand Watch ListLoading...Sign in to see your user information My eBayExpand My eBay Summary Recently Viewed Bids/Offers Watchlist Purchase History Buy Again Selling Saved Searches Saved Sellers My Garage Messages Co

#### now, identify and print to screen the title, price, and shipping price of each item.

In [52]:
def get_shipping_price(el):

        try:
            sp_text = el.text
            if sp_text.lower() =='free shipping':
                return 0
            return float(re.sub(r"([+, $  a-z \r\n])", "" , sp_text))
        except:
            return 0
 
def get_cost_title(el): 
    try:
        return float((re.findall(r"[$]\d+\.\d+|[$]\d+", el))[0].lstrip('$'))
    except:
        return 0
    
def get_price(el): 
    try:
        return float(el.lstrip('$'))
    except:
        return 0    

In [53]:
#"price": i.find('span', attrs={'class':'s-item__price'}).text
products = []

for l in range(1,11):
    soupn = BeautifulSoup(loadString(f=folder_to_Save+"amazon_gift_card_"+ str(l) +".htm"),'lxml')
    for idx,i in enumerate(soupn.find_all('div' ,attrs={'class':'s-item__info clearfix'})):  
        if i.find('span', attrs={'role':'heading'}).text !='Shop on eBay':
            products.append({"title": i.find('div', attrs={'class':'s-item__title'}).text, \
                             "price": get_price(i.find('span', attrs={'class':'s-item__price'}).text), \
                             "shipping": get_shipping_price(i.find('span', attrs={'class':'s-item__shipping s-item__logisticsCost'})),\
                             'costs_in_title': get_cost_title(i.find('span', attrs={'role':'heading'}).text)
                            })
  

In [10]:
len(products)

600

In [11]:
products

[{'title': 'New ListingAmazon Gift Cards ($200 / New Physical Cards) **Free Shipping**',
  'price': 195.0,
  'shipping': 0,
  'costs_in_title': 200.0},
 {'title': 'Amazon $200 Gift Card, Brand New Unopened Physical Card, Free Shipping (US ONLY)',
  'price': 202.5,
  'shipping': 0,
  'costs_in_title': 200.0},
 {'title': 'New ListingAmazon $100 Gift Card, Brand New, Package Unopened',
  'price': 93.05,
  'shipping': 5.25,
  'costs_in_title': 100.0},
 {'title': 'Amazon $100 Gift Card, Brand New, Package Unopened',
  'price': 98.0,
  'shipping': 5.25,
  'costs_in_title': 100.0},
 {'title': 'New ListingAmazon Gift Card $36',
  'price': 35.0,
  'shipping': 0,
  'costs_in_title': 36.0},
 {'title': 'New ListingAmazon Gift Card $30',
  'price': 29.0,
  'shipping': 0,
  'costs_in_title': 30.0},
 {'title': 'New ListingAmazon US Gift Card $200 Value Physical/Mailed Delivery',
  'price': 195.0,
  'shipping': 0,
  'costs_in_title': 200.0},
 {'title': 'New ListingAmazon Gift Card $95',
  'price': 93.

#### using RegEx, identify and print to screen gift cards that sold above face value (use RegEx to extract the value of a gift card from its title when possible).

In [12]:
products[5]['title']

'New ListingAmazon Gift Card $30'

In [13]:
(re.split(r"([$])" , products[5]['title']))

['New ListingAmazon Gift Card ', '$', '30']

In [14]:
(re.findall(r"([$0-9]+)" , 'New Listing$50.6 Amazon Gift Card'))

['$50', '6']

In [15]:
((re.findall(r"[$]\d+\.\d+|[$]\d+", 'New Listing $50 Amazon Gift Card')))

['$50']

####  Next compare a gift card’s value to its price + shipping (free shipping should be treated as 0).  If value < price + shipping, then a gift card sells above face value.

In [54]:
count_sold_above_face_value = 0

for i in products:
    #print(i['shipping'],i['price'], i['costs_in_title'],i['costs_in_title']< i['shipping']+i['price'])
    
    count_sold_above_face_value = count_sold_above_face_value + (i['costs_in_title']< i['shipping']+i['price'])

In [55]:
count_sold_above_face_value

290

####  fraction of Amazon gift cards selling above face value. Why ?

In [17]:
count_sold_above_face_value

290

In [18]:
len(products)

600

In [20]:
count_sold_above_face_value/len(products)

0.48833333333333334

In [19]:
print("The % of amazon gift cards that sell above face value is " + str(round(count_sold_above_face_value*100/len(products),2))+"%")

The % of amazon gift cards that sell above face value is 48.33%


In [20]:
print("The value is 48.8% which is high because often the customers wouldn't want to buy anything above the value of the card. Even then, there is high % of cards which still are listed above face value. One aspect that can be contributed to it is since they are amazon gift cards and are sold on a 3rd party platform ebay, ebay might want to have it's commission and hence includes additional charges in the form of shipping and prices.")

The value is 48.8% which is high because often the customers wouldn't want to buy anything above the value of the card. Even then, there is high % of cards which still are listed above face value. One aspect that can be contributed to it is since they are amazon gift cards and are sold on a 3rd party platform ebay, ebay might want to have it's commission and hence includes additional charges in the form of shipping and prices.
